# MEE-MVR desalination

A multistage evaporator with mechanical vapor recompression has been modeled in Pyomo. This notebook demonstrates how to build the model, specify inlet conditions and obtain CAPEX and OPEX post optimization.
The MEE-MVR model is taken from Onishi's 2017 paper on multi-effect evaporators [1].

We will begin with the relevant imports:

1) Pyomo for developing the model 

2) `degrees_of_freedom` from IDAES core to analyse the degrees of freedom

3) `make_mee_mvr_model` from PARETO

[1] V. C. Onishi, et.al. “Shale gas flowback water desalination: Single vs multiple effect evaporation with vapor recompression cycle and thermal integration,” Desalination, vol. 404, pp. 230–248, Feb. 2017

In [ ]:
#####################################################################################################
# PARETO was produced under the DOE Produced Water Application for Beneficial Reuse Environmental
# Impact and Treatment Optimization (PARETO), and is copyright (c) 2021-2024 by the software owners:
# The Regents of the University of California, through Lawrence Berkeley National Laboratory, et al.
# All rights reserved.
#
# NOTICE. This Software was developed under funding from the U.S. Department of Energy and the U.S.
# Government consequently retains certain rights. As such, the U.S. Government has been granted for
# itself and others acting on its behalf a paid-up, nonexclusive, irrevocable, worldwide license in
# the Software to reproduce, distribute copies to the public, prepare derivative works, and perform
# publicly and display publicly, and to permit others to do so.
#####################################################################################################

import pyomo.environ as pyo
from idaes.core.util.model_statistics import degrees_of_freedom
from pareto.models_extra.desalination_models.mee_mvr import make_mee_mvr_model

The inputs to the MEE-MVR process are:

1) `N_evap` - An integer indicating the number of evaporator stages (default = 1)

2) `inputs_variables` - A boolean indicating if the feed and salinity to the desalination unit are modeled as variables or parameters (the default value of False corresponds to parameters)

In this notebook, we will always assume our inputs are parameters (`inputs_variables` = False). 

In [ ]:
N_evap = 1
m = make_mee_mvr_model(N_evap=N_evap, inputs_variables=False)

To specify the inlet feed flow and salinity, we set the flow_feed and salt_feed to the desired values. In this example we set feed flowrate to 10 kg/s and feed salinity to 100 g/kg.

In [ ]:
m.flow_feed = 10
m.salt_feed = 100

Parameters such as the feed temperature, outlet specification of brine stream and the maximum compression ratio of the compressor can be user defined. 
In this case the feed temperature is set to 25 C and concentration of the brine stream is set to 250 g/kg and maximum compression ratio is 4.

In [ ]:
m.feed_temperature = 25
m.salt_outlet_spec = 250
m.CR_max = 4

The number of degrees of freedom can be checked using the `degrees_of_freedom` function in IDAES.

In [ ]:
print("DOF = ", degrees_of_freedom(m))

The MEE-MVR model has 2 degrees of freedom. We will first fix these two degrees of freedom to run a simulation. The evaporator area is fixed to 300 sq. meter and compressor capacity is fixed to 2000 hp.

In [ ]:
m.each_evaporator_area[0].fix(300)
m.compressor_capacity.fix(2000)
print("DOF = ", degrees_of_freedom(m))

# Solving the simulation
We will use the open source non-linear solver IPOPT available through Pyomo. The following block instantiates IPOPT and solves the MEE-MVR simulation

In [ ]:
ipopt = pyo.SolverFactory("ipopt")
ipopt.solve(m, tee=True)

The objective value can be obtained from the solved model. 

In [ ]:
print("Objective function =", pyo.value(m.obj))
print("preheater area = ", pyo.value(m.preheater_area))

# Optimization
Now we unfix the degrees of freedom to solve the optimization problem. We will minimize the total annualized cost and let the optimizer decide the optimal evaporator area, and compressor capacity

In [ ]:
m.each_evaporator_area[0].unfix()
m.compressor_capacity.unfix()
print("DOF = ", degrees_of_freedom(m))
ipopt.solve(m, tee=True)

In [ ]:
print("Objective function = ", pyo.value(m.obj))
print("preheater area = ", pyo.value(m.preheater_area))
print("Evaporator area = ", pyo.value(m.each_evaporator_area[0]))
print("Compressor capacity = ", pyo.value(m.compressor_capacity))